# Data Loading/Preprocessing

Downloads

In [4]:
%pip install gensim
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
from sklearn.model_Word2Vec_selection import train_test_split
from gensim.model_Word2Vecs import Word2Vec
import numpy as np
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import re
import string

# nltk stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

#loading the dataset
df = pd.read_csv('../dataset/emotions.csv')

# tokenization and stopwords
def clean_and_tokenize(text):
    return [word for word in text.split() if word.lower() not in stop_words]

df['tokenized'] = df['text'].apply(clean_and_tokenize)

# Train Word2Vec
model_Word2Vec = Word2Vec(sentences=df['tokenized'], vector_size=100, window=5, min_count=1, workers=4)

# Function to create sentence vectors by averaging word vectors
def sentence_vector(sentence):
    #creates word vectors. Checks if the word exists in the w2v model_Word2Vecs vocabulary
    vectors = [model_Word2Vec.wv[word] for word in sentence if word in model_Word2Vec.wv]
    #if if vectors is not empty put in the average vector
    if vectors:
        return np.mean(vectors, axis=0)
    #otherwise put in zeros
    else:
        return np.zeros(model_Word2Vec.vector_size)

X = np.array([sentence_vector(sentence) for sentence in df['tokenized']])

y = df['label']

#One hot encode the labels
y_encoded = to_categorical(y, num_classes=6)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

X_train_padded = pad_sequences(X_train, maxlen=20)
X_test_padded = pad_sequences(X_test, maxlen=20)

[nltk_data] Downloading package stopwords to /home/mizhap/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
max_length = df['tokenized'].apply(lambda x: len(x)).max()
print(max_length)

79


In [48]:
X_train[3]

array([-0.27580506,  0.88342899,  0.00157142,  0.58292574,  0.93032235,
        0.01089578,  0.42091909,  0.36756992, -0.64774066,  0.22177424,
       -0.69263035, -0.55858701,  0.62164718,  0.16056427,  0.05239609,
       -0.16093135,  0.59210461, -0.46135303,  0.6523816 ,  0.31251249,
        0.22542621, -0.07573688, -0.17835747,  0.04734053,  0.53203708,
       -0.04820093, -0.4895753 , -0.26316565, -0.12227771, -0.38372055,
        0.41837701,  0.43012854, -0.10650428, -0.03093025, -0.3338652 ,
        0.11545113, -0.89732176, -0.41437379, -0.26221266, -0.18717544,
        0.27794462, -0.50342852,  0.15163994, -0.09936377,  1.02685571,
       -0.43805838,  0.14258055, -0.5789572 , -0.84315854, -0.52676469,
        0.87412834, -0.67753685, -0.35771728,  0.55785424, -0.88567603,
       -0.66216666,  0.4759753 ,  0.5579192 , -0.67180985, -0.13322593,
       -0.19987462,  0.46587709,  0.15967521, -0.26305005, -0.73773807,
        0.00853632,  0.06831495, -0.11329015, -0.14831172,  0.47

# RNN Model

In [62]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Reshape

num_classes = 6

rnn = Sequential()
rnn.add(Reshape((100, 1), input_shape=(100,)))
rnn.add(SimpleRNN(units=64))  # Add your desired RNN layer (e.g., SimpleRNN)
rnn.add(Dense(units=num_classes, activation='softmax'))  # Add output layer with appropriate activation

# Compile the model
rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10
batch_size = 256

# Train the model
rnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Evaluate the model
loss, accuracy = rnn.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


/home/mizhap/.local/lib/python3.11/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.5172 - loss: 1.2701 - val_accuracy: 0.6119 - val_loss: 1.0397
Epoch 2/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.6182 - loss: 1.0233 - val_accuracy: 0.5963 - val_loss: 1.0907
Epoch 3/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 39s 37ms/step - accuracy: 0.6352 - loss: 0.9741 - val_accuracy: 0.6418 - val_loss: 0.9706
Epoch 4/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 37s 35ms/step - accuracy: 0.6512 - loss: 0.9332 - val_accuracy: 0.5932 - val_loss: 1.0760
Epoch 5/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 37s 35ms/step - accuracy: 0.6542 - loss: 0.9207 - val_accuracy: 0.6055 - val_loss: 1.0420
Epoch 6/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 37s 36ms/step - accuracy: 0.6567 - loss: 0.9133 - val_accuracy: 0.6392 - val_loss: 0.9622
Epoch 7/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 37s 35ms/step - accuracy: 0.6669 - loss: 0.8876 - val_accuracy: 0.5363 - val_loss: 1.1751
Epoch 8/10
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 38s 36ms/step - accuracy: 0.6506 -

In [64]:
rnn.save('emotions_rnn.keras')

In [65]:
from keras.models import load_model

model = load_model('emotions_rnn.keras')

/home/mizhap/.local/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
